In [1]:
from resolve import resolve 
resolve()
from widgets import dashboard
import ipywidgets as iw 
from IPython import display

username = input("Please enter your username: ")
display(username)
dashboard.run(username)

TypeError: 'module' object is not callable

In [1]:
from tqdm.notebook import tqdm
import time
for i in tqdm([1, 2, 3]):
    time.sleep(0.11)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from 